# SupermarketDetection with YOLOv8 (Training Notebook)
## Zoren Martinez (UniPD)

Note: This notebook was executer on Google Colab

## Imports and setup

In [1]:
!pip install torch==2.2.0+cu118 torchvision==0.17.0+cu118 torchaudio==2.2.0+cu118 --index-url https://download.pytorch.org/whl/cu118

!pip install ultralytics

# Downgrade NumPy to a version compatible with the installed PyTorch
# If you're in Colab, runtime reboot must be done
!pip install numpy==1.26.4


import torch, os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Setup complete. Using torch {torch.__version__} on {device}")

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_versio

Setup complete. Using torch 2.2.0+cu118 on cuda


## Dataset download
Source: [https://www.kaggle.com/datasets/thedatasith/sku110k-annotations](https://www.kaggle.com/datasets/thedatasith/sku110k-annotations)

In [1]:
!pip install kagglehub
import kagglehub
kagglehub.login()



Kaggle credentials set.


In [2]:
path = kagglehub.dataset_download("thedatasith/sku110k-annotations")

Using Colab cache for faster access to the 'sku110k-annotations' dataset.


In [11]:
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sku110k-annotations


In [12]:
!mkdir datasets
!cp -r {path} /content/datasets
!ls /content/datasets

sku110k-annotations


## Training

In [1]:
correct_yaml_path = "/content/datasets/data_sku110k.yaml"
correct_image_path = "/content/datasets/sku110k-annotations/SKU110K_fixed/images"

yaml_content = f"""
path: {correct_image_path}
train: train
val: val
test: test

nc: 1
names: ['object']
"""

with open(correct_yaml_path, "w") as f:
    f.write(yaml_content)

print(f"File YAML in '{correct_yaml_path}' corretto con successo.")

File YAML in '/content/datasets/data_sku110k.yaml' corretto con successo.


In [4]:
from ultralytics import YOLO
import time

RUN_NAME = time.strftime("%Y%m%d-%H%M%S")
print("Run name:", RUN_NAME)

model = YOLO("yolov8s.pt")

model.train(
    data="/content/datasets/data_sku110k.yaml",
    imgsz=640,
    epochs=20,
    batch=16,
    project="runs/train",
    name=RUN_NAME,
    cache=True
)

Run name: 20251015-161859
Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.2.0+cu118 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/data_sku110k.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=20251015-161859, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a0aa84e0500>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## Validation

In [5]:
model.val(
    data="/content/datasets/data_sku110k.yaml",
    imgsz=640,
    project="runs/val",
    name=RUN_NAME
)


Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.2.0+cu118 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 124.9±39.0 MB/s, size: 1195.5 KB)
val: Scanning /content/datasets/sku110k-annotations/SKU110K_fixed/labels/val.cache... 584 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 584/584 959.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 37/37 0.8it/s 47.9s
                   all        584      90456      0.854      0.775      0.851      0.512
Speed: 1.1ms preprocess, 11.5ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to /content/runs/val/20251015-161859


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a0a42183d40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
from google.colab import files

best_model_path = f"/content/runs/train/{RUN_NAME}/weights/best.pt"
last_model_path = f"/content/runs/train/{RUN_NAME}/weights/last.pt"

files.download(best_model_path)
files.download(last_model_path)

!zip -r runs_{RUN_NAME}.zip /content/runs/
files.download(f"runs_{RUN_NAME}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: content/runs/ (stored 0%)
  adding: content/runs/val/ (stored 0%)
  adding: content/runs/val/20251015-161859/ (stored 0%)
  adding: content/runs/val/20251015-161859/confusion_matrix.png (deflated 32%)
  adding: content/runs/val/20251015-161859/confusion_matrix_normalized.png (deflated 36%)
  adding: content/runs/val/20251015-161859/val_batch0_labels.jpg (deflated 3%)
  adding: content/runs/val/20251015-161859/val_batch2_labels.jpg (deflated 2%)
  adding: content/runs/val/20251015-161859/val_batch0_pred.jpg (deflated 3%)
  adding: content/runs/val/20251015-161859/BoxF1_curve.png (deflated 17%)
  adding: content/runs/val/20251015-161859/BoxPR_curve.png (deflated 20%)
  adding: content/runs/val/20251015-161859/val_batch1_pred.jpg (deflated 2%)
  adding: content/runs/val/20251015-161859/val_batch1_labels.jpg (deflated 2%)
  adding: content/runs/val/20251015-161859/BoxR_curve.png (deflated 17%)
  adding: content/runs/val/20251015-161859/BoxP_curve.png (deflated 20%)
  adding: cont

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>